In [1]:
from fastai.text import *   # Quick access to NLP functionality

In [32]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/ds/DataScience/home/ademyanchuk/.fastai/data/imdb_sample')

In [33]:
path.ls()

[PosixPath('/home/ds/DataScience/home/ademyanchuk/.fastai/data/imdb_sample/texts.csv'),
 PosixPath('/home/ds/DataScience/home/ademyanchuk/.fastai/data/imdb_sample/models')]

In [34]:
df = pd.read_csv(path/'texts.csv', header=None)
df.head()

,0,1,2
0,label,text,is_valid
1,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
2,positive,This is a extremely well-made film. The acting...,False
3,negative,Every once in a long while a movie will come a...,False
4,positive,Name just says it all. I watched this movie wi...,False


In [5]:
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')

In [13]:
learn = language_model_learner(data_lm)

In [6]:
pretrained_fnames = ['lstm_wt103', 'itos_wt103']

In [7]:
learn = language_model_learner(data_lm, pretrained_fnames=pretrained_fnames)

In [8]:
moms = (0.8,0.7)

In [9]:
learn.unfreeze()

In [10]:
learn.fit_one_cycle(4, slice(1e-2), moms=moms)

epoch,train_loss,valid_loss,accuracy
1,4.618361,3.895687,0.292520
2,4.312834,3.740356,0.297549
3,4.061804,3.704552,0.300940
4,3.888322,3.701622,0.300606


### Merge positive and negative examplles for ruSentEval

In [3]:
data_path = Path('../data/ruSentEval/')

In [5]:
data_path.ls()

[PosixPath('../data/ruSentEval/negative.csv'),
 PosixPath('../data/ruSentEval/positive.csv')]

In [10]:
pos_df = pd.read_csv(data_path/'positive.csv', sep=';', header=None)

In [13]:
pos_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,label
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0,positive
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2,positive
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0,positive
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0,positive
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1,positive


In [12]:
pos_df['label'] = 'positive'

In [17]:
pos_df.rename(columns={3:'text'}, inplace=True)

In [18]:
neg_df = pd.read_csv(data_path/'negative.csv', sep=';', header=None)

In [23]:
neg_df.head()

,0,1,2,text,4,5,6,7,8,9,10,11,label
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2,negative
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0,negative
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0,negative
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0,negative
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0,negative


In [21]:
neg_df['label'] = 'negative'

In [22]:
neg_df.rename(columns={3:'text'}, inplace=True)

In [28]:
all_df = pd.concat([pos_df.loc[:, ['label', 'text']], neg_df.loc[:, ['label', 'text']]], ignore_index=True)

In [44]:
all_df['is_valid'] = np.random.choice([False, True], len(all_df), p=[0.9, 0.1])

In [45]:
all_df.head()

,label,text,is_valid
0,positive,"@first_timee хоть я и школота, но поверь, у на...",False
1,positive,"Да, все-таки он немного похож на него. Но мой ...",False
2,positive,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,False
3,positive,"RT @digger2912: ""Кто то в углу сидит и погибае...",False
4,positive,@irina_dyshkant Вот что значит страшилка :D\nН...,False


In [48]:
all_df.loc[all_df.label == 'positive', 'is_valid'].sum()

11529

In [49]:
all_df.loc[all_df.label == 'negative', 'is_valid'].sum()

11216

In [50]:
all_df.to_csv(data_path/'texts.csv', sep=',', index=False, header=True)